<a href="https://colab.research.google.com/github/wvls123/colab/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_19_%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8F_Weweb_%2B_GPT_%D0%94%D0%B5%D0%BD%D1%8C_2_%D0%9A%D0%BE%D0%BB%D0%B0%D0%B1_%D0%B4%D0%B5%D0%BC%D0%BE_%D1%81%D0%B0%D0%B9%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

НЕЙРО - КОНСУЛЬТАНТ ЧУДО ЧЕРДАЧОК по VC


### Версия докрученная вручную с голосовым переключением в вевебе

In [ ]:
# ====== WEWEB BACKEND (FINAL) ======

!pip install -q openai tiktoken faiss-cpu gTTS flask flask-cors requests

import os, io, time, json, sqlite3, requests, faiss, numpy as np, tempfile, base64
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pathlib import Path
from gtts import gTTS
from getpass import getpass
from openai import OpenAI
from threading import Thread

api_key = getpass("Введите OpenAI API ключ: ").strip()
if not api_key:
    raise ValueError("Ключ не введён.")
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

PORT = 5001
LLM_MODEL = "gpt-4.1"
EMBED_MODEL = "text-embedding-3-small"
WHISPER_MODEL = "whisper-1"
CHUNK_SIZE_TOKENS = 5000
TOP_K = 7
EMBED_DIM = 1536

KNOWLEDGE_BASE_URL = "https://docs.google.com/document/d/1SoaYlYnjz6uAUOmQknRQy-dEcpq8E5GE5b1WqCfpgZM/export?format=txt"
SYSTEM_PROMPT_URL = "https://docs.google.com/document/d/191VR2bZ2VEFGNtnyA2OcigaT2vYy-1nuRLCqEKf160I/export?format=txt"

INDEX_DIR = Path("/content/rag_index")
INDEX_DIR.mkdir(parents=True, exist_ok=True)
FAISS_INDEX_PATH = INDEX_DIR / "faiss.index"
DOCS_META_PATH = INDEX_DIR / "docs_meta.json"
MEMORY_DB_PATH = INDEX_DIR / "memory.sqlite"

app = Flask(__name__)
CORS(app)

def init_memory():
    conn = sqlite3.connect(MEMORY_DB_PATH)
    conn.execute("""CREATE TABLE IF NOT EXISTS messages (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id TEXT, role TEXT, text TEXT, timestamp REAL
    )""")
    conn.commit()
    conn.close()

def save_message(uid, role, text):
    conn = sqlite3.connect(MEMORY_DB_PATH)
    conn.execute("INSERT INTO messages (user_id, role, text, timestamp) VALUES (?,?,?,?)",
                 (uid, role, text, time.time()))
    conn.commit()
    conn.close()

def get_messages(uid, limit=30):
    conn = sqlite3.connect(MEMORY_DB_PATH)
    # Берем последние сообщения и сохраняем их естественный порядок
    rows = conn.execute("""
        SELECT role, text FROM messages
        WHERE user_id=?
        ORDER BY id ASC  -- ← Меняем на ASC или убираем разворот
        LIMIT ?
    """, (uid, limit)).fetchall()
    conn.close()
    return rows

init_memory()

def clear_history_all():
    conn = sqlite3.connect(MEMORY_DB_PATH)
    conn.execute("DELETE FROM messages")
    conn.commit()
    conn.close()
    print("[CLEAR] Вся история очищена")

def clear_history_user(uid):
    conn = sqlite3.connect(MEMORY_DB_PATH)
    conn.execute("DELETE FROM messages WHERE user_id=?", (uid,))
    conn.commit()
    conn.close()
    print(f"[CLEAR] История {uid} очищена")

def trim_history_user(uid, limit=50):
    conn = sqlite3.connect(MEMORY_DB_PATH)
    rows = conn.execute("SELECT id FROM messages WHERE user_id=? ORDER BY id DESC", (uid,)).fetchall()
    if len(rows) > limit:
        ids_to_delete = [r[0] for r in rows[limit:]]
        conn.executemany("DELETE FROM messages WHERE id=?", [(i,) for i in ids_to_delete])
        conn.commit()
        print(f"[TRIM] История {uid} обрезана до {limit}")
    conn.close()

def download_text(url):
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        print(f"[OK] Загружено {len(r.text)} символов")
        return r.text
    except Exception as e:
        print(f"[ERROR] Загрузка: {e}")
        return ""

def split_into_chunks(text, size=CHUNK_SIZE_TOKENS):
    raw_chunks = [ch.strip() for ch in text.split("#") if ch.strip()]
    final_chunks = []
    for ch in raw_chunks:
        while len(ch) > size:
            part = ch[:size]
            final_chunks.append(part)
            ch = ch[size:]
        if ch:
            final_chunks.append(ch)
    print(f"[CHUNKS] Создано {len(final_chunks)} чанков")
    return final_chunks

def load_docs(url):
    txt = download_text(url)
    if not txt:
        return []
    chunks = split_into_chunks(txt)
    return [{"id": f"chunk_{i}", "text": ch, "src": url} for i, ch in enumerate(chunks)]

def load_prompt(url):
    t = download_text(url).strip()
    print(f"[PROMPT] Загружен промпт ({len(t)} символов)")
    return t if t else "Ты вежливый ассистент."

def embed_texts(texts):
    clean_texts = [t for t in texts if isinstance(t, str) and t.strip()]
    if not clean_texts:
        return np.zeros((0, EMBED_DIM), np.float32)
    embeddings = []
    for i in range(0, len(clean_texts), 10):
        batch = clean_texts[i:i+10]
        try:
            response = client.embeddings.create(model=EMBED_MODEL, input=batch)
            vectors = [np.array(item.embedding, dtype=np.float32) for item in response.data]
            embeddings.extend(vectors)
        except Exception as e:
            print(f"[ERROR] Embed: {e}")
            continue
    if not embeddings:
        return np.zeros((0, EMBED_DIM), np.float32)
    arr = np.vstack(embeddings)
    print(f"[EMBED] Получено {arr.shape[0]} векторов")
    return arr

def build_index(docs):
    embs = embed_texts([d["text"] for d in docs])
    faiss.normalize_L2(embs)
    index = faiss.IndexFlatIP(EMBED_DIM)
    index.add(embs)
    faiss.write_index(index, str(FAISS_INDEX_PATH))
    json.dump(docs, open(DOCS_META_PATH, "w", encoding="utf8"), ensure_ascii=False)
    print(f"[INDEX] Построен индекс: {len(docs)} чанков")

def load_index():
    if FAISS_INDEX_PATH.exists() and DOCS_META_PATH.exists():
        return faiss.read_index(str(FAISS_INDEX_PATH)), json.load(open(DOCS_META_PATH, encoding="utf8"))
    return None, None

print("Загружаю базу знаний...")
docs = load_docs(KNOWLEDGE_BASE_URL)
if docs:
    build_index(docs)
index, meta = load_index()

def retrieve(query, topk=TOP_K):
    if not index or not meta:
        return []
    q = embed_texts([query])
    faiss.normalize_L2(q)
    _, I = index.search(q, topk)
    chosen = [meta[i] for i in I[0] if i < len(meta)]
    print(f"[SEARCH] Найдено {len(chosen)} чанков")
    return chosen

def compose_prompt(txt, uid):
    sys_prompt = load_prompt(SYSTEM_PROMPT_URL)
    rel = retrieve(txt)
    hist = get_messages(uid)
    parts = [{"role": "system", "content": sys_prompt}]
    if hist:
        for r, t in hist[-10:]:
            parts.append({"role": r, "content": t})
    context_text = "\n\n".join([d["text"] for d in rel]) if rel else ""
    user_input = f"{txt}\n\nКонтекст:\n{context_text}" if context_text else txt
    parts.append({"role": "user", "content": user_input})
    print(f"[PROMPT] История: {len(hist)}, контекст: {len(rel)}")
    return parts

def generate_answer(messages):
    try:
        completion = client.chat.completions.create(
            model=LLM_MODEL,
            messages=messages,
            temperature=0.5,
            max_tokens=700
        )
        answer = completion.choices[0].message.content.strip()
        print(f"[OK] Ответ: {len(answer)} символов")
        return answer
    except Exception as e:
        print(f"[ERROR] LLM: {e}")
        return "Произошла ошибка при генерации ответа."

def transcribe_audio(data):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_in:
        tmp_in.write(data)
        tmp_in.flush()
        try:
            with open(tmp_in.name, "rb") as f:
                tr = client.audio.transcriptions.create(model=WHISPER_MODEL, file=f)
            text = tr.text.strip()
            print(f"[STT] Распознано: {text[:60]}...")
            return text
        except Exception as e:
            print(f"[ERROR] STT: {e}")
            return ""
        finally:
            os.remove(tmp_in.name)

def make_tts(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_out:
        try:
            gTTS(text=text, lang="ru").save(tmp_out.name)
            print("[TTS] Готово")
            return tmp_out.name
        except Exception as e:
            print(f"[ERROR] TTS: {e}")
            return ""

@app.route("/health")
def health():
    return jsonify({"ok": True, "time": time.time()})

@app.route("/webhook/text", methods=["POST"])
def webhook_text():
    try:
        data = request.get_json(force=True)
        uid = data.get("user_id", "anon")
        text = data.get("text", "").strip()
        tts_flag = data.get("tts") in [True, "true", "True"]

        if not text:
            return jsonify({"error": "text is empty"}), 400

        print(f"\n[TEXT] {uid}: {text[:50]}... | TTS={tts_flag}")

        save_message(uid, "user", text)
        messages = compose_prompt(text, uid)
        answer = generate_answer(messages)
        save_message(uid, "assistant", answer)

        response = {
            "answer": answer,
            "audio_base64": None
        }

        if tts_flag:
            path = make_tts(answer)
            if path and os.path.exists(path):
                with open(path, "rb") as f:
                    audio_b64 = base64.b64encode(f.read()).decode()
                    response["audio_base64"] = f"data:audio/mpeg;base64,{audio_b64}"
                os.remove(path)
                print(f"[TTS] Добавлено аудио в base64")

        return jsonify(response)

    except Exception as e:
        print(f"[ERROR] /webhook/text: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/webhook/audio", methods=["POST"])
def webhook_audio():
    try:
        uid = request.form.get("user_id", "anon")
        tts_flag = request.form.get("tts") in [True, "true", "True"]
        f = request.files.get("file")

        if not f:
            return jsonify({"error": "no audio file"}), 400

        print(f"\n[AUDIO] {uid} | TTS={tts_flag}")

        text = transcribe_audio(f.read())
        save_message(uid, "user", text)
        messages = compose_prompt(text, uid)
        answer = generate_answer(messages)
        save_message(uid, "assistant", answer)

        response = {
            "transcription": text,
            "answer": answer,
            "audio_base64": None
        }

        if tts_flag:
            path = make_tts(answer)
            if path and os.path.exists(path):
                with open(path, "rb") as f:
                    audio_b64 = base64.b64encode(f.read()).decode()
                    response["audio_base64"] = f"data:audio/mpeg;base64,{audio_b64}"
                os.remove(path)
                print(f"[TTS] Добавлено аудио в base64")

        return jsonify(response)

    except Exception as e:
        print(f"[ERROR] /webhook/audio: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/reload_kb", methods=["POST"])
def reload_kb():
    global KNOWLEDGE_BASE_URL, SYSTEM_PROMPT_URL, index, meta
    try:
        data = request.get_json(force=True)
        KNOWLEDGE_BASE_URL = data.get("kb_url", KNOWLEDGE_BASE_URL)
        SYSTEM_PROMPT_URL = data.get("prompt_url", SYSTEM_PROMPT_URL)
        docs = load_docs(KNOWLEDGE_BASE_URL)
        if docs:
            build_index(docs)
            index, meta = load_index()
        return jsonify({"status": "reloaded", "chunks": len(meta) if meta else 0})
    except Exception as e:
        print(f"[ERROR] /reload_kb: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/clear_history", methods=["POST"])
def clear_history():
    try:
        data = request.get_json(force=True)
        uid = data.get("user_id")
        mode = data.get("mode", "user")
        limit = data.get("limit")
        if mode == "all":
            clear_history_all()
            return jsonify({"status": "ok", "cleared": "all"})
        elif limit:
            trim_history_user(uid, int(limit))
            return jsonify({"status": "ok", "trimmed_to": limit})
        else:
            clear_history_user(uid)
            return jsonify({"status": "ok", "cleared": uid})
    except Exception as e:
        print(f"[ERROR] /clear_history: {e}")
        return jsonify({"error": str(e)}), 500

print("Освобождаю порт...")
!fuser -k {PORT}/tcp 2>/dev/null || true

print(f"Запускаю Flask на порту {PORT}...")
Thread(target=lambda: app.run(host="0.0.0.0", port=PORT, debug=False), daemon=True).start()

time.sleep(5)
print("Flask запущен!")

print("Создаю туннель...")
!wget -q -O cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb > /dev/null 2>&1
!cloudflared tunnel --url http://127.0.0.1:{PORT} --no-autoupdate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.8 MB/s eta 0:00:00
Загружаю базу знаний...
[OK] Загружено 38127 символов
[CHUNKS] Создано 64 чанков
[EMBED] Получено 64 векторов
[INDEX] Построен индекс: 64 чанков
Освобождаю порт...
Запускаю Flask на порту 5001...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit


Flask запущен!
Создаю туннель...
2025-11-19T16:30:17Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-19T16:30:17Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-19T16:30:21Z INF +--------------------------------------------------------------------------------------------+
2025-11-19T16:30:21Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-19T16:30:21Z INF |  https://space-eau-im

INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 16:31:57] "OPTIONS /clear_history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 16:31:57] "POST /clear_history HTTP/1.1" 200 -


[CLEAR] История 4f122ea2-a5a4-4ed9-9d6c-2ca36ce40439 очищена


...

[OK] Ответ: 163 символов


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 16:32:39] "OPTIONS /webhook/text HTTP/1.1" 200 -



[TEXT] 4f122ea2-a5a4-4ed9-9d6c-2ca36ce40439: 20 человек, из них 10 детей... | TTS=False
[OK] Загружено 15307 символов
[PROMPT] Загружен промпт (15307 символов)
[EMBED] Получено 1 векторов
[SEARCH] Найдено 7 чанков
[PROMPT] История: 5, контекст: 7


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 16:32:48] "POST /webhook/text HTTP/1.1" 200 -


[OK] Ответ: 934 символов
